In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from deep_translator import GoogleTranslator
import csv
import re

In [ ]:
df = pd.read_csv("historic_data.csv", delimiter=",")
df.head()

In [ ]:
len(df)

# Unique Users

In [ ]:
df_users_unique = df.drop_duplicates(subset=['participantUuid'])
print("Number of unique users: ", len(df_users_unique))

In [ ]:
df_users_unique.head()

# Purpose

In [ ]:
df_purpose = df[df['purpose'].notnull()]
df_purpose.head()

In [ ]:
df_purpose_unique = df_purpose.drop_duplicates(subset=['purpose']).reset_index(drop=True)
df_purpose_unique['purpose']

In [ ]:
len(df_purpose)

In [ ]:
pattern = re.compile(r'Procra*')
len(df_purpose[df_purpose['purpose'].str.contains(pattern, regex=True) ])

In [ ]:
len(df)

In [ ]:
## This loops through the unique Purpose entries and translates them all to english
# note that a bunch are already in english, but some are in german
# file_name = 'purpose_translations.csv'

# # temp = pd.DataFrame(0, index=np.arange(len(df_purpose_unique['purpose'])), columns=range(1))

# with open (file_name, 'w', newline='', encoding='utf-8-sig') as csv_file:
#     csv_writer = csv.writer(csv_file)
#     for index, row in df_purpose_unique.iloc[1086:-1].iterrows():
#         try:
#             temp[0].iloc[index] = GoogleTranslator(source='auto', target='en').translate(row['purpose'])
#             new_row = [row['purpose'], temp[0].iloc[index]]
#         except:
#             new_row = [row['purpose'], row['purpose']]
        
#         csv_writer.writerow(new_row)

# temp

# Apps

In [ ]:
df.groupby('app')['app'].count().sort_values(ascending=False).iloc[-25:-1]
# # df = df.sort_values(['participantUuid', 'timestamp'], ascending=[True, True])

In [ ]:
# Create a summary of the 'app' column
summary_apps = df['app'].value_counts().reset_index()
summary_apps.columns = ['app', 'count']

# Calculate percentage and add a new column
summary_apps['percentage'] = (summary_apps['count'] / summary_apps['count'].sum()) * 100
summary_apps['Cumulative'] = summary_apps['percentage'].cumsum()
# summary_apps.tail(20)



In [ ]:
# # Generate the pie chart
# plt.figure(figsize=(8, 6))
# plt.pie(summary_apps['count'], labels=summary_apps['app'], autopct='%1.1f%%', startangle=140)
# plt.title('App Distribution')
# plt.axis('equal')  # Equal aspect ratio ensures that the pie chart is circular.
# plt.show()

In [ ]:
summary_apps.to_csv('app_usage.csv', index=False)

In [ ]:
# Create a dictionary for mapping
mapping_dict = {
    'instagram': 'Social Media',
    'twitter': 'Social Media',
    'youtube': 'Social Media',
    'tikTok': 'Social Media',
    'facebook': 'Social Media',
    'reddit': 'Social Media',
    'snapchat': 'Social Media',
    'apollo': 'Social Media',
    'whastapp': 'Communication',
    'whastapp': 'Communication',
    'linkedin': 'Social Media',
    'telegram': 'Communication',
    'safari': 'Browser',
    'mail': 'Communication',
    'discord': 'Communication',
    'imessage': 'Communication',
    'gmail': 'Communication',
    'grinder': 'Dating',
    'chrome': 'Browser',
    'wechat': 'Communication',
    'facebookMessenger': 'Communication',
    'tinder': 'Dating',
    'amazon': 'Shopping',
    'bumble': 'Dating',
    'hinge': 'Dating',
    'jodel': 'Social Media',
    'appStore': 'Utility',
    'sparkMail': 'Communication',
    'newYorkTimes': 'News & Entertainment',
    'hackerNews': 'News & Entertainment',
    'netflix': 'News & Entertainment',
    'heyEmail': 'Communication',
    'outlook': 'Communication',
    'pornhub': 'News & Entertainment',
    'weibo': 'Social Media',
    'twitch': 'Social Media',
    'duckDuckGoBrowser': 'Dating',
    'whatsAppBusiness': 'Dating',
    'douban': 'Social Media',
    'spiegel': 'News & Entertainment',
    'tagesschau': 'News & Entertainment',
    'appleNews': 'News & Entertainment',
    'tweetBot': 'Social Media',
    'pinterest': 'Social Media',
    'clashRoyale': 'Gaming',
    'brave': 'Browser',
    'stocks': 'Finance',
    'reeder': 'News & Entertainment',
    'signal': 'Communication',
    'vinted': 'Shopping',
    'twitterrific': 'Social Media',
    'slack': 'Communication',
    'zeitOnline': 'News & Entertainment',
    'narwhal': 'Social Media',
    'bild': 'News & Entertainment',
    'beReal': 'Social Media',
    'tumblr': 'Social Media',
    'chessCom': 'Gaming',
    'binance': 'Finance',
    'firefox': 'Browser',
    'kuCoin': 'Finance',
    'google': 'Browser',
    'toot': 'Social Media',
    'webtoon': 'News & Entertainment',
    'mastodon': 'Social Media',
    'scruff': 'Dating',
    'tradeRepublic': 'Finance'
    
}


# Map the values in the 'platform' column using the dictionary
df['app_mapped'] = df['app'].map(mapping_dict).fillna('Other')

# Create a summary of the 'app' column
summary_app_categories = df['app_mapped'].value_counts().reset_index()
summary_app_categories.columns = ['app_mapped', 'count']

# Calculate percentage and add a new column
summary_app_categories['percentage'] = (summary_app_categories['count'] / summary_app_categories['count'].sum()) * 100
summary_app_categories['Cumulative'] = summary_app_categories['percentage'].cumsum()


In [ ]:
summary_app_categories.to_csv('app_categories.csv', index=False)

In [ ]:
user_list = df_users_unique['userIndex']
df_users_unique['has_social_media'] = False

for user in user_list:
    df_users_unique['has_social_media'].loc[df_users_unique['userIndex'] == user] = (df[df['userIndex'] == 1]['app_mapped'] == 'Social Media').any()



In [ ]:
100* (len(df_users_unique[df_users_unique['has_social_media'] == True]) / len(df_users_unique['userIndex']))

# Time Difference Conversions

In [ ]:
df = df.sort_values(['timestamp'], ascending=[True])
df['realSecondsDifference'] = (pd.to_datetime(df['timestamp']) - pd.to_datetime(df['timestamp'].iloc[0])).dt.total_seconds()

In [ ]:
df_users_adjust = df.sort_values(['participantUuid', 'timestamp'], ascending=[True, True]).drop_duplicates(['participantUuid']).reset_index(drop=True)
df_users_adjust['toSubtractDate'] = df_users_adjust['timestamp']

In [ ]:
df = df.merge(df_users_adjust[['participantUuid', 'toSubtractDate']], on='participantUuid')
df.head()

In [ ]:
# Calculate the time differences as accurately as possible for each unit (seconds, minutes, hours, days, weeks). Convert to int so that everything is floored (e.g. everything from week 0.0 to 0.9 is week 0, etc.)

df['realMinutesDifference'] = (df['realSecondsDifference'] / 60).astype(int)
df['realHoursDifference'] = (df['realSecondsDifference'] / 3600).astype(int)
df['realDaysDifference'] = (df['realSecondsDifference'] / 86400).astype(int)
df['realWeeksDifference'] = (df['realSecondsDifference'] / 608400).astype(int)

df['dateSinceStart'] = pd.to_datetime(df['timestamp']) - pd.to_datetime(df['toSubtractDate'])
df['secondsSinceStart'] = ((pd.to_datetime(df['timestamp']) - pd.to_datetime(df['toSubtractDate'])).dt.total_seconds())
df['minutesSinceStart'] = (df['secondsSinceStart'] / 60).astype(int)
df['hoursSinceStart'] = (df['secondsSinceStart'] / 3600).astype(int)
df['daysSinceStart2'] = (df['secondsSinceStart'] / 86400).astype(int)
df['daysSinceStart'] = ((pd.to_datetime(df['timestamp']) - pd.to_datetime(df['toSubtractDate'])).dt.days).astype(int)
df['weeksSinceStart'] = (df['daysSinceStart'] / 7).astype(int)

In [ ]:
df['day_of_week'] = pd.to_datetime(df['timestamp']).dt.strftime('%A')

# Resolution

In [ ]:
df['resolution'].unique()

In [ ]:
df['continuedToApp'] = np.where(df['resolution'] == 'openedApp', 'openedApp', np.where(df['resolution'] == 'closedApp', 'notAnIntervention','didNotOpenApp'))
df.head()


# Distribution of time differences - calculate what is a break

In [ ]:
df = df.sort_values(['participantUuid', 'timestamp'], ascending=[True, True])
df['minuteDiffReverse'] = df['minutesSinceStart'].diff(periods=1).fillna(0) # this makes the first one NA, subtracts each row from the row before
df['minuteDiffForward'] = df['minutesSinceStart'].diff(periods=-1).fillna(0) # this makes the last one NA, subtracts the next row from each row
df['minuteDiffForward'] = abs(df['minuteDiffForward'])
df['hourDiffReverse'] = (df['minuteDiffReverse']/60).astype(int)
df['hourDiffForward'] = (df['minuteDiffForward']/60).astype(int)
df['dayDiffReverse'] = (df['hourDiffReverse']/24).astype(int)
df['dayDiffForward'] = (df['hourDiffForward']/24).astype(int)
df['weekDiffReverse'] = (df['dayDiffReverse']/7).astype(int)
df['weekDiffForward'] = (df['dayDiffForward']/7).astype(int)
df['userDiffNext'] = df['participantUuid'] != df['participantUuid'].shift(-1)
df['userDiffPrev'] = df['participantUuid'] != df['participantUuid'].shift(1)

df.head()

In [ ]:
df_diff_distribution = df[df['userDiffPrev'] != True]
df_diff_distribution = df_diff_distribution[df_diff_distribution['resolution'] != 'closedApp']

In [ ]:
print("Min: ", df_diff_distribution['hourDiffReverse'].min())
print("Max: ", df_diff_distribution['hourDiffReverse'].max())
print("Mean: ", df_diff_distribution['hourDiffReverse'].mean())
print("Median: ", df_diff_distribution['hourDiffReverse'].median())
print("Std Dev: ", df_diff_distribution['hourDiffReverse'].std())
print("Two Std Dev above the mean (hours): ", df_diff_distribution['hourDiffReverse'].mean() + 2*df_diff_distribution['hourDiffReverse'].std())
print("Two Std Dev above the mean (days): ", (df_diff_distribution['hourDiffReverse'].mean() + 2*df_diff_distribution['hourDiffReverse'].std())/24)
print("Two Std Dev above the mean (weeks): ", (df_diff_distribution['hourDiffReverse'].mean() + 2*df_diff_distribution['hourDiffReverse'].std())/(7*24))

mean_timeDiff_hours = df_diff_distribution['hourDiffReverse'].mean()
std_timeDiff_hours = df_diff_distribution['hourDiffReverse'].std()
break_cutoff_timeDiff_hours = mean_timeDiff_hours + 2*std_timeDiff_hours

break_cutoff_timeDiff_hours

In [ ]:
print("Min: ", df_diff_distribution['minuteDiffReverse'].min())
print("Max: ", df_diff_distribution['minuteDiffReverse'].max())
print("Mean: ", df_diff_distribution['minuteDiffReverse'].mean())
print("Median: ", df_diff_distribution['minuteDiffReverse'].median())
print("Std Dev: ", df_diff_distribution['minuteDiffReverse'].std())
print("Two Std Dev above the mean (minutes): ", df_diff_distribution['minuteDiffReverse'].mean() + 2*df_diff_distribution['minuteDiffReverse'].std())
print("Two Std Dev above the mean (hours): ", (df_diff_distribution['minuteDiffReverse'].mean() + 2*df_diff_distribution['minuteDiffReverse'].std())/60)
print("Two Std Dev above the mean (days): ", (df_diff_distribution['minuteDiffReverse'].mean() + 2*df_diff_distribution['minuteDiffReverse'].std())/(60*24))

mean_timeDiff_minutes = df_diff_distribution['minuteDiffReverse'].mean()
std_timeDiff_minutes = df_diff_distribution['minuteDiffReverse'].std()
break_cutoff_timeDiff_minutes = mean_timeDiff_minutes + 2*std_timeDiff_minutes

break_cutoff_timeDiff_minutes

# Labeling Breaks

In [ ]:
df['lifeStatus'] = np.where(((~df['userDiffNext']) & (~df['userDiffPrev']) & (abs(df['minuteDiffForward']) > break_cutoff_timeDiff_minutes) & (abs(df['minuteDiffReverse']) > break_cutoff_timeDiff_minutes)), 'Return/Break',
                                np.where(((~df['userDiffNext']) & (~df['userDiffPrev']) & (abs(df['minuteDiffForward']) > break_cutoff_timeDiff_minutes)), 'Break','Alive'))

df.loc[df['lifeStatus'].shift() == 'Break', 'lifeStatus'] = 'Return'

In [ ]:
df_noclose = df[df['resolution'] != 'closedApp'].copy()

In [ ]:
df_noclose['lifeStatus'] = np.where(((~df_noclose['userDiffNext']) & (~df_noclose['userDiffPrev']) & (abs(df_noclose['minuteDiffForward']) > break_cutoff_timeDiff_minutes) & (abs(df_noclose['minuteDiffReverse']) > break_cutoff_timeDiff_minutes)), 'Return/Break',
                                        np.where(((~df_noclose['userDiffNext']) & (~df_noclose['userDiffPrev']) & (abs(df_noclose['minuteDiffForward']) > break_cutoff_timeDiff_minutes)), 'Break', 'Alive'))

df_noclose.loc[df_noclose['lifeStatus'].shift() == 'Break', 'lifeStatus'] = 'Return'

In [ ]:
print("Alive entries: ", len(df_noclose[df_noclose['lifeStatus'] == 'Alive']))
print("Break entries: ", len(df_noclose[df_noclose['lifeStatus'] == 'Break']))
print("Return/Break entries: ", len(df_noclose[df_noclose['lifeStatus'] == 'Return/Break']))
print("Return entries: ", len(df_noclose[df_noclose['lifeStatus'] == 'Return']))
print("Dropout entries: ", len(df_noclose[df_noclose['lifeStatus'] == 'Dropout']))
print("Participants with at least one break: ", len(df_noclose[df_noclose['lifeStatus'] == 'Break']['participantUuid'].unique()))
print("Unique participants: ", len(df_noclose['participantUuid'].unique()))

In [ ]:
print(len(df[df['lifeStatus'] == 'Alive']))
print(len(df[df['lifeStatus'] == 'Break']))
print(len(df[df['lifeStatus'] == 'Return/Break']))
print(len(df[df['lifeStatus'] == 'Return']))
print(len(df[df['lifeStatus'] == 'Dropout']))
print("Unique participants: ", len(df['participantUuid'].unique()))

In [ ]:
df.to_csv('data_processed.csv', index=False)

In [ ]:
df_noclose.to_csv('data_processed_noclose.csv', index=False)

In [ ]:
df_hasclose = df[df['resolution'] == 'closedApp'].copy()
users_without_close_enabled = df['participantUuid'].unique().size - df_hasclose['participantUuid'].unique().size
users_without_close_enabled

# Time in study, users in study at x time

## First, without considering the result of the action


In [ ]:
user_max_hours = df_noclose.groupby('userIndex')['hoursSinceStart'].max().reset_index(name='max_hours')

# Create a list of hours from 0 to the maximum of max_hours
max_hour = user_max_hours['max_hours'].max()
hours = list(range(max_hour + 1))

# Initialize a list to store the count of users for each hour
hourly_counts = []

# Iterate through the hours and count the number of users for each hour
for hour in hours:
    count = (user_max_hours['max_hours'] >= hour).sum()
    hourly_counts.append(count)

# Create the final dataframe
df_users_remaining_hours = pd.DataFrame({'hoursSinceStart': hours, 'user_count': hourly_counts})

# Print the result dataframe
print(df_users_remaining_hours)


In [ ]:
user_max_days = df_noclose.groupby('userIndex')['daysSinceStart'].max().reset_index(name='max_days')

# Create a list of hours from 0 to the maximum of max_hours
max_day = user_max_days['max_days'].max()
days = list(range(max_day + 1))

# Initialize a list to store the count of users for each hour
daily_counts = []

# Iterate through the hours and count the number of users for each hour
for day in days:
    count = (user_max_days['max_days'] >= day).sum()
    daily_counts.append(count)

# Create the final dataframe
df_users_remaining_days = pd.DataFrame({'daysSinceStart': days, 'user_count': daily_counts})

# Print the result dataframe
print(df_users_remaining_days)


In [ ]:
user_max_weeks = df_noclose.groupby('userIndex')['weeksSinceStart'].max().reset_index(name='max_weeks')

# Create a list of hours from 0 to the maximum of max_hours
max_week = user_max_weeks['max_weeks'].max()
weeks = list(range(max_week + 1))

# Initialize a list to store the count of users for each hour
weekly_counts = []

# Iterate through the hours and count the number of users for each hour
for week in weeks:
    count = (user_max_weeks['max_weeks'] >= week).sum()
    weekly_counts.append(count)

# Create the final dataframe
df_users_remaining_weeks = pd.DataFrame({'weeksSinceStart': weeks, 'user_count': weekly_counts})

# Print the result dataframe
print(df_users_remaining_weeks)


In [ ]:
df_entries_per_hour = df_noclose.groupby(['hoursSinceStart'])['userIndex'].count().reset_index(name='total_interactions')
df_entries_per_hour['std_interactions'] = df_noclose.groupby(['hoursSinceStart','userIndex'])['userIndex'].count().reset_index(name='total_interactions').groupby('hoursSinceStart')['total_interactions'].agg(['std']).reset_index()['std']
df_entries_per_user_hours = df_users_remaining_hours.merge(df_entries_per_hour, on='hoursSinceStart', how='left')
df_entries_per_user_hours['total_interactions'].fillna(method='ffill', inplace=True)
df_entries_per_user_hours['std_interactions'].fillna(0, inplace=True)
df_entries_per_user_hours['interactions_per_user'] = df_entries_per_user_hours['total_interactions'] / df_entries_per_user_hours['user_count']
df_entries_per_user_hours['std_error_interactions'] = df_entries_per_user_hours['std_interactions'] / np.sqrt(df_entries_per_user_hours['user_count'])


df_entries_per_day = df_noclose.groupby(['daysSinceStart'])['userIndex'].count().reset_index(name='total_interactions')
df_entries_per_day['std_interactions'] = df_noclose.groupby(['daysSinceStart','userIndex'])['userIndex'].count().reset_index(name='total_interactions').groupby('daysSinceStart')['total_interactions'].agg(['std']).reset_index()['std']
df_entries_per_user_days = df_users_remaining_days.merge(df_entries_per_day, on='daysSinceStart', how='left')
df_entries_per_user_days['total_interactions'].fillna(method='ffill', inplace=True)
df_entries_per_user_days['std_interactions'].fillna(0, inplace=True)
df_entries_per_user_days['interactions_per_user'] = df_entries_per_user_days['total_interactions'] / df_entries_per_user_days['user_count']
df_entries_per_user_days['std_error_interactions'] = df_entries_per_user_days['std_interactions'] / np.sqrt(df_entries_per_user_days['user_count'])


df_entries_per_week = df_noclose.groupby(['weeksSinceStart'])['userIndex'].count().reset_index(name='total_interactions')
df_entries_per_week['std_interactions'] = df_noclose.groupby(['weeksSinceStart','userIndex'])['userIndex'].count().reset_index(name='total_interactions').groupby('weeksSinceStart')['total_interactions'].agg(['std']).reset_index()['std']
df_entries_per_user_weeks = df_users_remaining_weeks.merge(df_entries_per_week, on='weeksSinceStart', how='left')
df_entries_per_user_weeks['total_interactions'].fillna(method='ffill', inplace=True)
df_entries_per_user_weeks['std_interactions'].fillna(0, inplace=True)
df_entries_per_user_weeks['interactions_per_user'] = df_entries_per_user_weeks['total_interactions'] / df_entries_per_user_weeks['user_count']
df_entries_per_user_weeks['std_error_interactions'] = df_entries_per_user_weeks['std_interactions'] / np.sqrt(df_entries_per_user_weeks['user_count'])


print(df_entries_per_user_hours.head())
print(df_entries_per_user_days.head())
print(df_entries_per_user_weeks.head())

In [ ]:
df_entries_per_user_hours.to_csv('data_users_remaining_hours.csv', index=False)
df_entries_per_user_days.to_csv('data_users_remaining_days.csv', index=False)
df_entries_per_user_weeks.to_csv('data_users_remaining_weeks.csv', index=False)

## Second, considering the result

In [ ]:
user_max_hours_result = df_noclose.groupby(['userIndex', 'continuedToApp'])['hoursSinceStart'].max().reset_index(name='max_hours')

# Create a list of hours from 0 to the maximum of max_hours
max_hour = user_max_hours_result['max_hours'].max()
hours = [i for i in range(max_hour) for _ in range(2)]

# Initialize a list to store the count of users for each hour
hourly_counts = []
continued_status = []
flag = True


# Iterate through the hours and count the number of users for each hour
for idx, hour in enumerate(hours):
    count = (user_max_hours['max_hours'] >= hour).sum()
    hourly_counts.append(count)
    if flag:
        continued_status.append('didNotOpenApp')
    else:
        continued_status.append('openedApp')
    flag = not flag

# Create the final dataframe
df_users_remaining_hours_result = pd.DataFrame({'hoursSinceStart': hours, 'user_count': hourly_counts, 'continuedToApp': continued_status})

# Print the result dataframe
print(df_users_remaining_hours_result)


In [ ]:
user_max_days_result = df_noclose.groupby(['userIndex', 'continuedToApp'])['daysSinceStart'].max().reset_index(name='max_days')

# Create a list of hours from 0 to the maximum of max_hours
max_day = user_max_days_result['max_days'].max()
days = [i for i in range(max_day) for _ in range(2)]

# Initialize a list to store the count of users for each hour
daily_counts = []
continued_status = []
flag = True

# Iterate through the hours and count the number of users for each hour
for idx, day in enumerate(days):
    count = (user_max_days['max_days'] >= day).sum()
    daily_counts.append(count)
    # continued_status.append(user_max_days_result['continuedToApp'].iloc[idx])
    if flag:
        continued_status.append('didNotOpenApp')
    else:
        continued_status.append('openedApp')
    flag = not flag

# Create the final dataframe
df_users_remaining_days_result = pd.DataFrame({'daysSinceStart': days, 'user_count': daily_counts, 'continuedToApp': continued_status})

# Print the result dataframe
print(df_users_remaining_days_result)


In [ ]:
user_max_weeks_result = df_noclose.groupby(['userIndex', 'continuedToApp'])['weeksSinceStart'].max().reset_index(name='max_weeks')

# Create a list of hours from 0 to the maximum of max_hours
max_week = user_max_weeks_result['max_weeks'].max()
weeks = [i for i in range(max_week) for _ in range(2)]

# Initialize a list to store the count of users for each hour
weekly_counts = []
continued_status = []
flag = True

# Iterate through the hours and count the number of users for each hour
for idx, week in enumerate(weeks):
    count = (user_max_weeks['max_weeks'] >= week).sum()
    weekly_counts.append(count)
    # continued_status.append(user_max_weeks_result['continuedToApp'].iloc[idx])
    if flag:
        continued_status.append('didNotOpenApp')
    else:
        continued_status.append('openedApp')
    flag = not flag

# Create the final dataframe
df_users_remaining_weeks_result = pd.DataFrame({'weeksSinceStart': weeks, 'user_count': weekly_counts, 'continuedToApp': continued_status})

# Print the result dataframe
print(df_users_remaining_weeks_result)

In [ ]:
# first, calculate the total number of interactions for each user at each timepoint for each value of continued to app
df_entries_per_hour_result = df_noclose.groupby(['hoursSinceStart', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions')
# next, calculate the standard deviation of all intaractions at each timepoint and continued to app value
df_entries_per_hour_result['std_interactions'] = df_noclose.groupby(['hoursSinceStart','userIndex', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions').groupby(['hoursSinceStart', 'continuedToApp'])['total_interactions'].agg(['std']).reset_index()['std']
# join the number of users remaining at each timepoint to the dataframe
df_entries_per_user_hours_result = df_users_remaining_hours_result.merge(df_entries_per_hour_result, on=['hoursSinceStart', 'continuedToApp'], how='left')
# remove duplicated columns (might not be necessary if you just join on all the common columns)
df_entries_per_user_hours_result = df_entries_per_user_hours_result.loc[:, ~df_entries_per_user_hours_result.columns.duplicated()]
df_entries_per_hour_result

In [ ]:

# first, calculate the total number of interactions for each user at each timepoint for each value of continued to app
df_entries_per_hour_result = df_noclose.groupby(['hoursSinceStart', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions')
# next, calculate the standard deviation of all intaractions at each timepoint and continued to app value
df_entries_per_hour_result['std_interactions'] = df_noclose.groupby(['hoursSinceStart','userIndex', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions').groupby(['hoursSinceStart', 'continuedToApp'])['total_interactions'].agg(['std']).reset_index()['std']
# join the number of users remaining at each timepoint to the dataframe
df_entries_per_user_hours_result = df_users_remaining_hours_result.merge(df_entries_per_hour_result, on=['hoursSinceStart', 'continuedToApp'], how='left')
# remove duplicated columns (might not be necessary if you just join on all the common columns)
df_entries_per_user_hours_result = df_entries_per_user_hours_result.loc[:, ~df_entries_per_user_hours_result.columns.duplicated()]
# fill 0s for NAs, it means there are hours with no interactions
df_entries_per_user_hours_result['total_interactions'].fillna(0, inplace=True)
# fill 0s for NAs, it means there are hours with no interactions or with only one user
df_entries_per_user_hours_result['std_interactions'].fillna(0, inplace=True)
# interactions per user at each time point = total interactions at that time point divided by number of users still in the study at that time point
df_entries_per_user_hours_result['interactions_per_user'] = df_entries_per_user_hours_result['total_interactions'] / df_entries_per_user_hours_result['user_count']
# standard error is the standard deviation divided by the square root of the N
df_entries_per_user_hours_result['std_error_interactions'] = df_entries_per_user_hours_result['std_interactions'] / np.sqrt(df_entries_per_user_hours_result['user_count'])
# calculate the squared error ratio for use later when propagating the errors to the percentage
df_entries_per_user_hours_result['squared_error_ratio'] = np.power(df_entries_per_user_hours_result['std_error_interactions'] / df_entries_per_user_hours_result['interactions_per_user'], 2)
# find the sum  total of interactions at each hour, independent of continued to app value
grouped_hour = df_entries_per_user_hours_result.groupby('hoursSinceStart')['interactions_per_user'].transform(sum)
# find the sum  total of interaction errors at each hour, independent of continued to app value
grouped_error_hour = df_entries_per_user_hours_result.groupby('hoursSinceStart')['std_error_interactions'].transform(sum)
#  percentage for each continued to app value is the number of interactions for that value divided by the total for that time point
df_entries_per_user_hours_result['percentage'] = 100 * (df_entries_per_user_hours_result['interactions_per_user'] / grouped_hour)
#  the percentage error is the percentage value divided by the square root of the sum of the squares of that particular error value ratio plus the sum of the error value ratios
df_entries_per_user_hours_result['std_error_percentage'] = df_entries_per_user_hours_result['percentage'] * np.sqrt(df_entries_per_user_hours_result['squared_error_ratio'] + np.power(grouped_error_hour,2) / np.power(grouped_hour,2) )

df_entries_per_day_result = df_noclose.groupby(['daysSinceStart', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions')
df_entries_per_day_result['std_interactions'] = df_noclose.groupby(['daysSinceStart','userIndex', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions').groupby(['daysSinceStart', 'continuedToApp'])['total_interactions'].agg(['std']).reset_index()['std']
df_entries_per_user_days_result = df_users_remaining_days_result.merge(df_entries_per_day_result, on=['daysSinceStart', 'continuedToApp'], how='left')
df_entries_per_user_days_result = df_entries_per_user_days_result.loc[:, ~df_entries_per_user_days_result.columns.duplicated()]
df_entries_per_user_days_result['total_interactions'].fillna(0, inplace=True)
df_entries_per_user_days_result['std_interactions'].fillna(0, inplace=True)
df_entries_per_user_days_result['interactions_per_user'] = df_entries_per_user_days_result['total_interactions'] / df_entries_per_user_days_result['user_count']
df_entries_per_user_days_result['std_error_interactions'] = df_entries_per_user_days_result['std_interactions'] / np.sqrt(df_entries_per_user_days_result['user_count'])
df_entries_per_user_days_result['squared_error_ratio'] = np.power(df_entries_per_user_days_result['std_error_interactions'] / df_entries_per_user_days_result['interactions_per_user'], 2)
grouped_day = df_entries_per_user_days_result.groupby('daysSinceStart')['interactions_per_user'].transform(sum)
grouped_error_day = df_entries_per_user_days_result.groupby('daysSinceStart')['std_error_interactions'].transform(sum)
df_entries_per_user_days_result['percentage'] = 100 * (df_entries_per_user_days_result['interactions_per_user'] / grouped_day)
df_entries_per_user_days_result['std_error_percentage'] = df_entries_per_user_days_result['percentage'] * np.sqrt(df_entries_per_user_days_result['squared_error_ratio'] + np.power(grouped_error_day,2) / np.power(grouped_day,2) )

df_entries_per_week_result = df_noclose.groupby(['weeksSinceStart', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions')
df_entries_per_week_result['std_interactions'] = df_noclose.groupby(['weeksSinceStart','userIndex', 'continuedToApp'])['userIndex'].count().reset_index(name='total_interactions').groupby(['weeksSinceStart', 'continuedToApp'])['total_interactions'].agg(['std']).reset_index()['std']
df_entries_per_user_weeks_result = df_users_remaining_weeks_result.merge(df_entries_per_week_result, on=['weeksSinceStart','continuedToApp'], how='left')
df_entries_per_user_weeks_result = df_entries_per_user_weeks_result.loc[:, ~df_entries_per_user_weeks_result.columns.duplicated()]
df_entries_per_user_weeks_result['total_interactions'].fillna(0, inplace=True)
df_entries_per_user_weeks_result['std_interactions'].fillna(0, inplace=True)
df_entries_per_user_weeks_result['interactions_per_user'] = df_entries_per_user_weeks_result['total_interactions'] / df_entries_per_user_weeks_result['user_count']
df_entries_per_user_weeks_result['std_error_interactions'] = df_entries_per_user_weeks_result['std_interactions'] / np.sqrt(df_entries_per_user_weeks_result['user_count'])
df_entries_per_user_weeks_result['squared_error_ratio'] = np.power(df_entries_per_user_weeks_result['std_error_interactions'] / df_entries_per_user_weeks_result['interactions_per_user'], 2)
grouped_week = df_entries_per_user_weeks_result.groupby('weeksSinceStart')['interactions_per_user'].transform(sum)
grouped_error_week = df_entries_per_user_weeks_result.groupby('weeksSinceStart')['std_error_interactions'].transform(sum)
df_entries_per_user_weeks_result['percentage'] = 100 * (df_entries_per_user_weeks_result['interactions_per_user'] / grouped_week)
df_entries_per_user_weeks_result['std_error_percentage'] = df_entries_per_user_weeks_result['percentage'] * np.sqrt(df_entries_per_user_weeks_result['squared_error_ratio'] + np.power(grouped_error_week,2) / np.power(grouped_week,2) )

print(df_entries_per_user_hours_result.head())
print(df_entries_per_user_days_result.head())
print(df_entries_per_user_weeks_result.head())


In [ ]:
df_entries_per_user_hours_result.to_csv('data_users_remaining_hours_result.csv', index=False)
df_entries_per_user_days_result.to_csv('data_users_remaining_days_result.csv', index=False)
df_entries_per_user_weeks_result.to_csv('data_users_remaining_weeks_result.csv', index=False)

# Just before Breaks

In [ ]:
# Assuming your original DataFrame is named 'df'
# Create a list of participantUuid who have at least one "Break"
participants_with_breaks = df_noclose[df_noclose['lifeStatus'] == 'Break']['participantUuid'].unique()

# Create a new DataFrame containing all entries for participants with breaks
filtered_df = df_noclose[df_noclose['participantUuid'].isin(participants_with_breaks)].copy()

# Sort the filtered DataFrame by 'participantUuid' and 'daysSinceStart'
filtered_df = filtered_df.sort_values(by=['participantUuid', 'hoursSinceStart'])

# Initialize a new column 'within_65_hours_before_break' with default value False
# filtered_df['within_65_hours_before_break'] = False

# Initialize a new column 'time_to_break' with default value NaN
filtered_df['time_to_break'] = float('nan')

# Iterate through each participant's data
for participant in participants_with_breaks:
    participant_data = filtered_df[filtered_df['participantUuid'] == participant]
    break_indices = participant_data[participant_data['lifeStatus'] == 'Break'].index
    prev_break_index = participant_data.index[0]

    for index in break_indices:
        prev_break_time = participant_data.loc[prev_break_index, 'hoursSinceStart']
        current_break_time = participant_data.loc[index, 'hoursSinceStart']

        within_range = (participant_data['hoursSinceStart'] >= max((current_break_time - break_cutoff_timeDiff_hours), prev_break_time)) & (participant_data['hoursSinceStart'] <= current_break_time)
        # filtered_df.loc[(filtered_df['participantUuid'] == participant) & within_range,'within_65_hours_before_break'] = True

        # within_range = (participant_data['hoursSinceStart'] >= prev_break_time) & (participant_data['hoursSinceStart'] <= current_break_time)
        
        filtered_df.loc[(filtered_df['participantUuid'] == participant) & within_range,'time_to_break'] = filtered_df.loc[(filtered_df['participantUuid'] == participant) & within_range,'hoursSinceStart'] - current_break_time

        prev_break_index = index

# Now, filtered_df contains the labeled entries within 65 hours before each "Break" row
# and the 'time_to_break' column indicating time to the next break
filtered_df = filtered_df[filtered_df['time_to_break'].notnull()]

In [ ]:
filtered_df.to_csv('data_breaks.csv', index=False)

# Just after Returns

In [ ]:

# Assuming your original DataFrame is named 'df'
# Create a list of participantUuid who have at least one "Return"
participants_with_returns = df_noclose[df_noclose['lifeStatus'] == 'Return']['participantUuid'].unique()

# Create a new DataFrame containing all entries for participants with returns
filtered_df = df_noclose[df_noclose['participantUuid'].isin(participants_with_returns)].copy()

# Sort the filtered DataFrame by 'participantUuid' and 'daysSinceStart'
filtered_df = filtered_df.sort_values(by=['participantUuid', 'hoursSinceStart'])

# Initialize a new column 'within_7_days_before_return' with default value False
# filtered_df['within_65_hours_after_return'] = False

# Initialize a new column 'time_to_return' with default value NaN
filtered_df['time_after_return'] = float('nan')

# Iterate through each participant's data
for participant in participants_with_returns:
    participant_data = filtered_df[filtered_df['participantUuid'] == participant]
    return_indices = participant_data[participant_data['lifeStatus'] == 'Return'].index
    prev_return_index = participant_data.index[0]

    for index in return_indices:
        current_return_time = participant_data.loc[prev_return_index, 'hoursSinceStart']
        next_return_time = participant_data.loc[index, 'hoursSinceStart']
    
        within_range = (participant_data['hoursSinceStart'] <= (current_return_time + break_cutoff_timeDiff_hours)) & (participant_data['hoursSinceStart'] >= current_return_time) & (participant_data['hoursSinceStart'] <= next_return_time)
        # filtered_df.loc[(filtered_df['participantUuid'] == participant) & within_range,'within_65_hours_after_return'] = True
        
        # within_range = (participant_data['hoursSinceStart'] >= current_return_time) & (participant_data['hoursSinceStart'] <= next_return_time)
        
        filtered_df.loc[(filtered_df['participantUuid'] == participant) & within_range,'time_after_return'] = filtered_df.loc[(filtered_df['participantUuid'] == participant) & within_range,'hoursSinceStart'] - current_return_time
        

        prev_return_index = index

# Now, filtered_df contains the labeled entries within 7 days before each "Return" row
# and the 'time_to_return' column indicating time to the next return
filtered_df = filtered_df[filtered_df['time_after_return'].notnull()]


In [ ]:
filtered_df.to_csv('data_returns.csv', index=False)

# Intervention

In [ ]:
df_intervention_unique = df.drop_duplicates(subset=['interventionType'])
df_intervention_unique = df_intervention_unique[df_intervention_unique['interventionType'].notnull()]
df_intervention_unique['interventionType']

In [ ]:
print(len(df_noclose['userIndex'].unique()))

df_interventions = df_noclose.groupby(['userIndex', 'interventionType'])['userIndex'].count()
print(df_interventions)


df_interventions_count = df_interventions.groupby('userIndex').count()
print(df_interventions_count)

In [ ]:
userIndexes_in_noclose = set(df_noclose['userIndex'].unique())

# Get unique userIndex values in df_interventions
userIndexes_in_interventions = set(df_interventions.index.get_level_values('userIndex').unique())

# Find userIndexes that are in df_noclose but not in df_interventions
userIndexes_not_in_interventions = userIndexes_in_noclose - userIndexes_in_interventions

# Convert the result to a list
userIndexes_not_in_interventions_list = list(userIndexes_not_in_interventions)

print(len(df_noclose['interventionType'].loc[df_noclose['userIndex'].isin([769, 636, 302])]))


# Resolution

In [ ]:
df['resolution'].unique()

In [ ]:
print('Opened the app', len(df[df['resolution'] == 'openedApp']))
print('Did not continue to the app', len(df[df['resolution'] == 'dismissedAppOpening']))
print('Structured intervention', len(df[df['resolution'] == 'dismissedByStructuredIntervention']))
print('Clicked on a healthy alternative', len(df[df['resolution'] == 'dismissedWithHealthyAlternative']))
print('Left the app (at the end)', len(df[df['resolution'] == 'closedApp']))

In [ ]:
len(df[df['resolution'] == 'dismissedByStructuredIntervention']['userIndex'].unique())

In [ ]:
_ = plt.hist(df['resolution'], bins=5)  # arguments are passed to np.histogram
plt.title("Resolution")
plt.show()

# Trend Testing

In [ ]:
import pymannkendall as mk

## Overall Use

In [ ]:
df_trend = df_entries_per_user_days.loc[df_entries_per_user_days['daysSinceStart'] <= 307]
df_trend_result = df_entries_per_user_days_result.loc[df_entries_per_user_days_result['daysSinceStart'] <= 307]

In [ ]:
result = mk.original_test(df_trend['interactions_per_user'])
result

In [ ]:
result = mk.original_test(df_trend_result[df_trend_result['continuedToApp'] == 'didNotOpenApp']['percentage'])
result

In [ ]:
result = mk.sens_slope(df_trend['interactions_per_user'])
result

In [ ]:
result = mk.sens_slope(df_trend_result[df_trend_result['continuedToApp'] == 'didNotOpenApp']['percentage'])
result

## First 48 hours

In [ ]:
df_trend_hours = df_entries_per_user_hours.loc[df_entries_per_user_hours['hoursSinceStart'] <= 48]
df_trend_hours_result = df_entries_per_user_hours_result.loc[df_entries_per_user_hours_result['hoursSinceStart'] <= 48]

In [ ]:
result = mk.original_test(df_trend_hours['interactions_per_user'])
result

In [ ]:
result = mk.original_test(df_trend_hours_result[df_trend_hours_result['continuedToApp'] == 'didNotOpenApp']['percentage'])
result

In [ ]:
result = mk.sens_slope(df_trend_hours['interactions_per_user'])
result

In [ ]:
result = mk.sens_slope(df_trend_hours_result[df_trend_hours_result['continuedToApp'] == 'didNotOpenApp']['percentage'])
result

In [ ]:
result = mk.hamed_rao_modification_test(df_trend_hours['interactions_per_user'])
result